# Get data and packages

In [1]:
%%capture
%%bash
pip install jupyter_contrib_nbextensions
pip install jupyter_nbextensions_configurator
jupyter contrib nbextension install --user
jupyter nbextensions_configurator enable --user

jupyter nbextension enable codefolding/main
jupyter nbextension enable scratchpad/main
jupyter nbextension enable execute_time/ExecuteTime
jupyter nbextension enable autosavetime/main
pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow


In [ ]:
%%capture
!pip install -U pip dask numpy fsspec>=0.3.3 tqdm pyarrow

In [ ]:
!aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet ../datasets/taxi_parquet

In [5]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.dask_utils import *
from src.config import repetitions

name = 'dask'
data_path = '../datasets/taxi_parquet/'
results_path = f"../results/{name}_1b.csv"
benchmarks = {}
print(f"test for {repetitions} repetitions")

test for 1 repetitions


# Benchmark

In [4]:
# Load data
data = read_file(None, data_path)
# data.pickup_datetime = dd.to_datetime(data.pickup_datetime)
# data['pickup_hour'] = data.pickup_datetime.dt.hour
# data = data.set_index('passenger_count')
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057928 with 18 columns


In [ ]:
benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
benchmarks['sum columns']= benchmark(sum_columns, data, repetitions=repetitions)
benchmarks['product columns']= benchmark(product_columns, data, repetitions=repetitions)
benchmarks['arithmetic operation']= benchmark(complicated_arithmetic_operation, data, repetitions=repetitions)
benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
print(f"cleaned {gc.collect()} mb")
benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
print(f"Done benchmarks on all data")

# filtered
filterd = filter_data(data)
del data

print(f"Prepare filtered data and deleted {gc.collect()} MB")
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered sum columns'] = benchmark(sum_columns , filterd, repetitions=repetitions)
benchmarks['filtered product_columns'] = benchmark(product_columns , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))
print(f"Done benchmarks on filterd data")


In [ ]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
results.head()

In [6]:
!aws s3 cp  ../results/dask_1b.csv s3://vaex-sagemaker-demo/benchmarks/dask_1b_results_mlm44xlarge.csv 

upload: ../results/dask_1b.csv to s3://vaex-sagemaker-demo/benchmarks/dask_1b_results_mlm44xlarge.csv


# Benchmark line by line

In [7]:
# Load data
data = read_file(None, data_path)
# data.pickup_datetime = dd.to_datetime(data.pickup_datetime)
# data['pickup_hour'] = data.pickup_datetime.dt.hour
# data = data.set_index('passenger_count')
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057928 with 18 columns


In [8]:
def persiste_results():
    get_results(benchmarks, name).to_csv(results_path)

In [9]:
benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
persiste_results()

In [10]:
benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
persiste_results()

In [11]:
benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
persiste_results()

In [12]:
benchmarks['sum columns']= benchmark(sum_columns, data, repetitions=repetitions)
persiste_results()

In [13]:
benchmarks['product columns'] = benchmark(product_columns, data, repetitions=repetitions)
persiste_results()

In [14]:
benchmarks['arithmetic operation'] = benchmark(complicated_arithmetic_operation, data, repetitions=repetitions)
persiste_results()

In [15]:
benchmarks['value counts'] = benchmark(value_counts, data, repetitions=repetitions)
persiste_results()

In [16]:
benchmarks['groupby statistics'] = benchmark(groupby_statistics, data, repetitions=repetitions)
persiste_results()

In [18]:
gc.collect()

567

In [ ]:
benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
persiste_results()

In [ ]:
print(f"cleaned {gc.collect()} mb")
benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
persiste_results()
print(f"Done benchmarks on all data")

In [ ]:
filterd = filter_data(data)
del data
print(f"Prepare filtered data and deleted {gc.collect()} MB")

In [ ]:
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered sum columns'] = benchmark(sum_columns , filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered complicated arithmetic operation'] = benchmark(complicated_arithmetic_operation, filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
persiste_results()

In [ ]:
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))
persiste_results()

In [ ]:
print(f"Done benchmarks on filterd data")